# Vogel's approximation
## For assignment1

In [17]:
import numpy as np
import pandas as pd

# store the names of plant locations
names=['Mexico','Canada','Chile','Frankfurt','Austin','Japan']

# demand is already known
d=np.array([3,2.6,16,20,26.4,11.9])

# capacity is already known
s=np.array([22,3.7,4.5,47,18.5,5])

# create cost matrix
# production cost
c_production=np.array([[92.63,93.25,112.31,73.34,89.15,149.24]])
c_production=np.repeat(c_production, len(d)).reshape(len(s),len(d))

# duties cost
c_duties=np.array([[0.6,0,0.5,0.095,0.045,0.06]])
c_duties=np.repeat(c_duties, len(s)).reshape(len(s),len(d),order="F")
c_duties[range(len(s)),range(len(s))]=0 # set diagonal to 0
c_duties=c_duties+1

# shipping cost
c_shipping=np.array([[0,11.4,7,11,11,14],
                     [11,0,9,11.5,6,13],
                     [7,10,0,13,10.4,14.3],
                     [10,11.5,12.5,0,11.2,13.3],
                     [10,6,11,10,0,12.5],
                     [14,13,12.5,14.2,13,0]])
# total cost
c=c_production*c_duties+c_shipping

In [24]:
# define VAM optimization function
def vam_opt(s,d,c,show_iter=True):
    
    '''
    s, the original supply vector
    d, the original demand vector
    c, the original cost matrix
    show_iter, if not 0, print the results after each iteration
    '''
    
    # declare local variables
    s0,d0,c0=s.copy(),d.copy(),c.copy()
    
    # declare the matrix to be optimized
    x0=np.zeros((len(s0), len(d0)))

    # use a large m to represent an infinite large
    m=c0.max()*1000000
    
    # start optimization
    num_iter=0
    while min(sum(s0),sum(d0))>0:
        # exclude optimized rows and columns
        a=np.repeat(s0!=0, len(d0)).reshape(c0.shape) # bool operator that save supply
        b=np.repeat(d0!=0, len(s0)).reshape(c0.shape,order="F") # bool operator that save demand
        c0[(a & b)!=True]=m # set optimized row and column to inf large number

        # calculate opportunity cost arrays
        minc_s=np.repeat(c0.min(axis=1), c0.shape[1]).reshape(c0.shape) # get minimum of supply
        opcost_s=c0-minc_s # subtract the minimum to save oppo cost
        opcost_s=np.sort(opcost_s, axis=1) # sort the opcost by row
        opcost_s=opcost_s[:,1] # get the second to last opcost

        # same steps for demand
        minc_d=np.repeat(c0.min(axis=0), c0.shape[0]).reshape(c0.shape,order="F")
        opcost_d=c0-minc_d
        opcost_d=np.sort(opcost_d, axis=0)
        opcost_d=opcost_d[1,:] # get the second to last opcost

        # find the index of s and d to be optimized
        maxcost=max(max(opcost_s),max(opcost_d))
        rows=opcost_s==maxcost
        cols=opcost_d==maxcost
        index1=np.repeat(rows, c0.shape[1]).reshape(c0.shape) # get row index
        index2=np.repeat(cols, c0.shape[0]).reshape(c0.shape,order="F") # get col index
        index=index1 | index2
        c_index=c0*index
        ctmp=c_index+(c_index==0)*c_index.max() # replace 0 by the max number, therefore can use min() to find the minimum positive

        i=np.where(ctmp==ctmp.min())[0][0]
        j=np.where(ctmp==ctmp.min())[1][0]

        # upgrade x,s,d
        x0[i,j]=min(s0[i],d0[j])  # x: shipping number, min of s and d
        s0[i]=s0[i]-x0[i,j]
        d0[j]=d0[j]-x0[i,j]
        
        num_iter+=1
        
        # output results after each iteration
        if show_iter:
            print('After %d iteration(s):' % num_iter)
            print('x%d=\n' % num_iter,x0)
            print('d%d=\n' % num_iter,d0)
            print('s%d=\n' % num_iter,s0)
            print()
    return x0

In [25]:
# define output farmatting function
def output_table(x):
    # x, the optimized matrix
    x_df=pd.DataFrame(x)
    x_df.columns,x_df.index=names,names
    x_df['Production Plan']=x_df.sum(axis=1)
    x_df['Capacity(%)']=(x_df['Production Plan']/s*100).round(2).astype(str)+'%'
    display(x_df)

### Q2 use all capacity

In [26]:
s_q2=s.copy()

In [27]:
x_q2=vam_opt(s=s_q2,d=d,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x_q2*c)/100))

After 1 iteration(s):
x1=
 [[3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
d1=
 [ 0.   2.6 16.  20.  26.4 11.9]
s1=
 [19.   3.7  4.5 47.  18.5  5. ]

After 2 iteration(s):
x2=
 [[ 3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
d2=
 [ 0.   2.6 16.   0.  26.4 11.9]
s2=
 [19.   3.7  4.5 27.  18.5  5. ]

After 3 iteration(s):
x3=
 [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]
d3=
 [ 0.   2.6 16.   0.  26.4  0. ]
s3=
 [19.   3.7  4.5 15.1 18.5  5. ]

After 4 iteration(s):
x4=
 [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   4.5  0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   

In [28]:
output_table(x_q2)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,0.0,0.0,0.0,3.2,0.0,6.2,28.18%
Canada,0.0,2.6,0.0,0.0,1.1,0.0,3.7,100.0%
Chile,0.0,0.0,4.5,0.0,0.0,0.0,4.5,100.0%
Frankfurt,0.0,0.0,11.5,20.0,3.6,11.9,47.0,100.0%
Austin,0.0,0.0,0.0,0.0,18.5,0.0,18.5,100.0%
Japan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%


### Q3 use 85% of all capacity

In [8]:
s_q3=s*0.85

In [9]:
x_q3=vam_opt(s=s_q3,d=d,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x_q3*c)/100))

After 1 iteration(s):
x1=
 [[3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
d1=
 [ 0.   2.6 16.  20.  26.4 11.9]
s1=
 [15.7    3.145  3.825 39.95  15.725  4.25 ]

After 2 iteration(s):
x2=
 [[ 3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
d2=
 [ 0.   2.6 16.   0.  26.4 11.9]
s2=
 [15.7    3.145  3.825 19.95  15.725  4.25 ]

After 3 iteration(s):
x3=
 [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]
d3=
 [ 0.   2.6 16.   0.  26.4  0. ]
s3=
 [15.7    3.145  3.825  8.05  15.725  4.25 ]

After 4 iteration(s):
x4=
 [[ 3.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     3.825  0.     0.     0.   ]
 [ 0.  

In [10]:
output_table(x_q3)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,0.0,4.125,0.0,5.880,0.0,13.005,59.11%
Canada,0.0,2.6,0.000,0.0,0.545,0.0,3.145,85.0%
Chile,0.0,0.0,3.825,0.0,0.000,0.0,3.825,85.0%
Frankfurt,0.0,0.0,8.050,20.0,0.000,11.9,39.950,85.0%
Austin,0.0,0.0,0.000,0.0,15.725,0.0,15.725,85.0%
Japan,0.0,0.0,0.000,0.0,4.250,0.0,4.250,85.0%


### Q4 not using Chile, use 90% of all capacity

In [11]:
s_q4=s*0.9
s_q4[[i=='Chile' for i in  names]]=0

In [12]:
x_q4=vam_opt(s=s_q4,d=d,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x_q4*c)/100))

After 1 iteration(s):
x1=
 [[3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
d1=
 [ 0.   2.6 16.  20.  26.4 11.9]
s1=
 [16.8   3.33  0.   42.3  16.65  4.5 ]

After 2 iteration(s):
x2=
 [[ 3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
d2=
 [ 0.   2.6 16.   0.  26.4 11.9]
s2=
 [16.8   3.33  0.   22.3  16.65  4.5 ]

After 3 iteration(s):
x3=
 [[ 3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0. 16. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
d3=
 [ 0.   2.6  0.   0.  26.4 11.9]
s3=
 [16.8   3.33  0.    6.3  16.65  4.5 ]

After 4 iteration(s):
x4=
 [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  16.  20.   0.   6.3]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. 

In [13]:
output_table(x_q4)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,0.0,0.0,0.0,9.02,1.1,13.12,59.64%
Canada,0.0,2.6,0.0,0.0,0.73,0.0,3.33,90.0%
Chile,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0%
Frankfurt,0.0,0.0,16.0,20.0,0.00,6.3,42.30,90.0%
Austin,0.0,0.0,0.0,0.0,16.65,0.0,16.65,90.0%
Japan,0.0,0.0,0.0,0.0,0.00,4.5,4.50,90.0%


### use the default production plan

In [14]:
x0=vam_opt(s=np.array([17.2,2.6,4.1,38,14,4]),d=d,c=c,show_iter=True)
print('Total cost: %.4f' % sum(sum(x0*c)/100))

After 1 iteration(s):
x1=
 [[3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
d1=
 [ 0.   2.6 16.  20.  26.4 11.9]
s1=
 [14.2  2.6  4.1 38.  14.   4. ]

After 2 iteration(s):
x2=
 [[ 3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
d2=
 [ 0.   2.6 16.   0.  26.4 11.9]
s2=
 [14.2  2.6  4.1 18.  14.   4. ]

After 3 iteration(s):
x3=
 [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]
d3=
 [ 0.   2.6 16.   0.  26.4  0. ]
s3=
 [14.2  2.6  4.1  6.1 14.   4. ]

After 4 iteration(s):
x4=
 [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   4.1  0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   

In [14]:
output_table(x0)

,Mexico,Canada,Chile,Frankfurt,Austin,Japan,Production Plan,Capacity(%)
Mexico,3.0,0.0,5.8,0.0,8.4,0.0,17.2,78.18%
Canada,0.0,2.6,0.0,0.0,0.0,0.0,2.6,70.27%
Chile,0.0,0.0,4.1,0.0,0.0,0.0,4.1,91.11%
Frankfurt,0.0,0.0,6.1,20.0,0.0,11.9,38.0,80.85%
Austin,0.0,0.0,0.0,0.0,14.0,0.0,14.0,75.68%
Japan,0.0,0.0,0.0,0.0,4.0,0.0,4.0,80.0%


### using the same production plan, VAM result is worse than the default plan shown in the reading material